In [1]:
# import subprocess
# subprocess.call(['sh', './train.sh'])

# Training

In [2]:
from itertools import chain
from s_clip_scripts.main import main
from s_clip_scripts.params import parse_args

fashion = False

if fashion:
    str_args = '''--model RN50 
            --pretrained openai 
            --train-data Fashion-ALL
            --label-ratio 0.15
            --val-data Fashion-ALL
            --keyword-path keywords/fashion/class-name.txt
            --lr 5e-5 
            --batch-size 64 
            --warmup 10 
            --epochs 10
            --precision amp 
            --method base 
            --seed 0 
            --active_learning
    '''
else:
    str_args = '''--model RN50 
            --pretrained openai 
            --train-data RS-ALL
            --label-ratio 0.2
            --val-data RS-ALL
            --imagenet-val RSICD-CLS \
            --keyword-path keywords/RS/class-name.txt
            --lr 5e-5 
            --batch-size 64 
            --warmup 10 
            --epochs 25
            --zeroshot-frequency 5 
            --precision amp 
            --method base 
            --seed 0
            --al_method mean
            --al-ratio 0.1
            --active_learning
    '''
str_args = str_args.split('\n')
str_args = [s.strip() for s in str_args]
str_args = [s.split(' ') for s in str_args]
str_args = list(chain(*str_args))
str_args = [s for s in str_args if len(s) > 0]
args = parse_args(str_args)

In [3]:
%%time
checkpoint_path = main(args) # Calls the main.py function of S-CLIP

Making log path: 2023_10_17-16_13_02-data_RS-ALL-ratio_0.2-model_RN50-method_base-keyword_none-AL_method_mean
Running with a single process on device cuda:0.
Getting data...
RS-ALL
Dataset length: 10911
Labeled size: 2182
Performing active learning.
Number of AL selected images: 1091
Number of labeled images: 2182


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1408/1408 [02:46<00:00,  8.46it/s]


sim shape (10911, 10911)
img shape (10911, 1024)
Using 1091 labeled images
cosine time: 101.6005539894104
(10911, 1091)
RS-ALL
RSICD-CLS
['Airport', 'BareLand', 'BaseballField', 'Beach', 'Bridge', 'Center', 'Church', 'Commercial', 'DenseResidential', 'Desert', 'Farmland', 'Forest', 'Industrial', 'Meadow', 'MediumResidential', 'Mountain', 'Park', 'Parking', 'Playground', 'Pond', 'Port', 'RailwayStation', 'Resort', 'River', 'School', 'SparseResidential', 'Square', 'Stadium', 'StorageTanks', 'Viaduct', 'playfields']
Data got.
Start epoch 0
Done training.
Done evaluating.
Start epoch 1
Done training.
Done evaluating.
Start epoch 2
Done training.
Done evaluating.
Start epoch 3
Done training.
Done evaluating.
Start epoch 4
Done training.
Done evaluating.
Start epoch 5
Done training.
Done evaluating.
Start epoch 6
Done training.
Done evaluating.
Start epoch 7
Done training.
Done evaluating.
Start epoch 8
Done training.
Done evaluating.
Start epoch 9
Done training.
Done evaluating.
Start epoch

# Eval

In [4]:
%%time
checkpoint = checkpoint_path # '2023_10_16-09_32_34-data_RS-ALL-ratio_0.1-model_RN50-method_base-keyword_none'
if 'Fashion' in checkpoint:
    zeroshot_datasets = ["Fashion200k-SUBCLS", "Fashion200k-CLS", "FashionGen-CLS", "FashionGen-SUBCLS", "Polyvore-CLS", ]
    retrieval_datasets = ["FashionGen", "Polyvore", "Fashion200k",]
else:
    zeroshot_datasets = ["RSICD-CLS", "UCM-CLS"] # "WHU-RS19", "RSSCN7", "AID" -> NOT WORKING bc of different data-loading workings
    retrieval_datasets = ["RSICD", "UCM", "Sydney"]

for dataset in zeroshot_datasets:
    str_args = ['--name', checkpoint, '--imagenet-val', dataset]
    args = parse_args(str_args)
    main(args)

for dataset in retrieval_datasets:
    str_args = ['--name', checkpoint, '--val-data', dataset]
    args = parse_args(str_args)
    main(args)

Running with a single process on device cuda:0.
=> resuming checkpoint '2023_10_17-16_13_02-data_RS-ALL-ratio_0.2-model_RN50-method_base-keyword_none-AL_method_mean/checkpoints/epoch_latest.pt' (epoch 25)
Getting data...
RSICD-CLS
['Airport', 'BareLand', 'BaseballField', 'Beach', 'Bridge', 'Center', 'Church', 'Commercial', 'DenseResidential', 'Desert', 'Farmland', 'Forest', 'Industrial', 'Meadow', 'MediumResidential', 'Mountain', 'Park', 'Parking', 'Playground', 'Pond', 'Port', 'RailwayStation', 'Resort', 'River', 'School', 'SparseResidential', 'Square', 'Stadium', 'StorageTanks', 'Viaduct', 'playfields']
Data got.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1152/1152 [00:07<00:00, 162.35it/s]


Running with a single process on device cuda:0.
=> resuming checkpoint '2023_10_17-16_13_02-data_RS-ALL-ratio_0.2-model_RN50-method_base-keyword_none-AL_method_mean/checkpoints/epoch_latest.pt' (epoch 25)
Getting data...
UCM-CLS
['agricultural', 'airplane', 'baseballdiamond', 'beach', 'buildings', 'chaparral', 'denseresidential', 'forest', 'freeway', 'golfcourse', 'harbor', 'intersection', 'mediumresidential', 'mobilehomepark', 'overpass', 'parkinglot', 'river', 'runway', 'sparseresidential', 'storagetanks', 'tenniscourt']
Data got.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2112/2112 [00:15<00:00, 137.49it/s]


Running with a single process on device cuda:0.
=> resuming checkpoint '2023_10_17-16_13_02-data_RS-ALL-ratio_0.2-model_RN50-method_base-keyword_none-AL_method_mean/checkpoints/epoch_latest.pt' (epoch 25)
Getting data...
RSICD
Data got.
Running with a single process on device cuda:0.
=> resuming checkpoint '2023_10_17-16_13_02-data_RS-ALL-ratio_0.2-model_RN50-method_base-keyword_none-AL_method_mean/checkpoints/epoch_latest.pt' (epoch 25)
Getting data...
UCM
Data got.
Running with a single process on device cuda:0.
=> resuming checkpoint '2023_10_17-16_13_02-data_RS-ALL-ratio_0.2-model_RN50-method_base-keyword_none-AL_method_mean/checkpoints/epoch_latest.pt' (epoch 25)
Getting data...
Sydney
Data got.
CPU times: user 47.1 s, sys: 9.28 s, total: 56.4 s
Wall time: 1min 7s
